In [1]:
from tensorflow.keras.datasets import boston_housing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
import tensorflow.keras as K
from tensorflow.keras import models
from tensorflow.keras import layers
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# Boston Housingデータセットのダウンロード
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [3]:
# データセットのデータの確認
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'ACE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
df = pd.DataFrame(train_data, columns=column_names)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,ACE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [4]:
# データの正規化
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

train_data = (train_data - mean) / std
test_data = (test_data - mean) / std

In [5]:
# モデルの定義
def build_model():
    # 同じモデルを複数回インスタンスかする必要があるため，
    # モデルをインスタンス化するための関数を使用
    model = models.Sequential()
    model.add(layers.Dense(64, 
                           activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,
                           activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [6]:
# k分割交差検証
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    # 検証データの準備: フォールドiのデータ
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    # 訓練データの準備: 残りのフォールドのデータ
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
    # Kerasモデルを構築(コンパイル済み)
    model = build_model()
    
    # モデルをサイレントモード（verbose=0）で適合
    model.fit(partial_train_data, partial_train_targets,
             epochs=num_epochs, batch_size=1)
    
    # モデルを検証データで評価
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
Train on 303 samples
Epoch 1/100
303/303 [==============================] - 3s 9ms/sample - loss: 229.3374 - mae: 11.8247
Epoch 2/100
303/303 [==============================] - 1s 3ms/sample - loss: 30.0752 - mae: 3.8432
Epoch 3/100
303/303 [==============================] - 1s 3ms/sample - loss: 23.2763 - mae: 3.3126
Epoch 4/100
303/303 [==============================] - 1s 3ms/sample - loss: 18.6245 - mae: 3.0066
Epoch 5/100
303/303 [==============================] - 1s 3ms/sample - loss: 17.2245 - mae: 2.8333
Epoch 6/100
303/303 [==============================] - 1s 3ms/sample - loss: 15.5193 - mae: 2.6623
Epoch 7/100
303/303 [==============================] - 1s 3ms/sample - loss: 15.1413 - mae: 2.5732
Epoch 8/100
303/303 [==============================] - 1s 3ms/sample - loss: 13.7203 - mae: 2.5150
Epoch 9/100
303/303 [==============================] - 1s 3ms/sample - loss: 13.6446 - mae: 2.4218
Epoch 10/100
303/303 [==============================] - 1s 3ms/sam

303/303 [==============================] - 1s 3ms/sample - loss: 5.1795 - mae: 1.4570
Epoch 84/100
303/303 [==============================] - 1s 3ms/sample - loss: 5.0137 - mae: 1.4578
Epoch 85/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.7926 - mae: 1.4384
Epoch 86/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.5183 - mae: 1.4550
Epoch 87/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.8272 - mae: 1.4422
Epoch 88/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.4297 - mae: 1.3494
Epoch 89/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.7722 - mae: 1.4310
Epoch 90/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.5856 - mae: 1.4330
Epoch 91/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.3111 - mae: 1.3894
Epoch 92/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.2342 - mae: 1.3751
Epoch 93/100
303/303 [=

303/303 [==============================] - 1s 3ms/sample - loss: 5.6993 - mae: 1.6189
Epoch 66/100
303/303 [==============================] - 1s 3ms/sample - loss: 5.5070 - mae: 1.4751
Epoch 67/100
303/303 [==============================] - 1s 3ms/sample - loss: 5.4660 - mae: 1.4832
Epoch 68/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.9182 - mae: 1.5414
Epoch 69/100
303/303 [==============================] - 1s 3ms/sample - loss: 5.7310 - mae: 1.5185
Epoch 70/100
303/303 [==============================] - 1s 3ms/sample - loss: 5.4554 - mae: 1.5118
Epoch 71/100
303/303 [==============================] - 1s 3ms/sample - loss: 5.5417 - mae: 1.4978
Epoch 72/100
303/303 [==============================] - 1s 3ms/sample - loss: 5.1982 - mae: 1.4995
Epoch 73/100
303/303 [==============================] - 1s 3ms/sample - loss: 5.4096 - mae: 1.4501
Epoch 74/100
303/303 [==============================] - 1s 3ms/sample - loss: 5.3645 - mae: 1.4363
Epoch 75/100
303/303 [=

303/303 [==============================] - 1s 3ms/sample - loss: 4.9538 - mae: 1.6220
Epoch 48/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.7292 - mae: 1.5303
Epoch 49/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.5300 - mae: 1.5651
Epoch 50/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.5792 - mae: 1.5116
Epoch 51/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.4725 - mae: 1.5027
Epoch 52/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.3281 - mae: 1.5246
Epoch 53/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.6306 - mae: 1.4915
Epoch 54/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.4802 - mae: 1.5231
Epoch 55/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.1620 - mae: 1.4520
Epoch 56/100
303/303 [==============================] - 1s 3ms/sample - loss: 4.2118 - mae: 1.5025
Epoch 57/100
303/303 [=

303/303 [==============================] - 1s 3ms/sample - loss: 7.6399 - mae: 1.7519
Epoch 30/100
303/303 [==============================] - 1s 3ms/sample - loss: 7.1544 - mae: 1.7866
Epoch 31/100
303/303 [==============================] - 1s 3ms/sample - loss: 6.9845 - mae: 1.7274
Epoch 32/100
303/303 [==============================] - 1s 3ms/sample - loss: 7.1136 - mae: 1.7024
Epoch 33/100
303/303 [==============================] - 1s 3ms/sample - loss: 6.5481 - mae: 1.6903
Epoch 34/100
303/303 [==============================] - 1s 3ms/sample - loss: 6.9717 - mae: 1.7334
Epoch 35/100
303/303 [==============================] - 1s 3ms/sample - loss: 6.9517 - mae: 1.7487
Epoch 36/100
303/303 [==============================] - 1s 3ms/sample - loss: 6.4899 - mae: 1.6896
Epoch 37/100
303/303 [==============================] - 1s 3ms/sample - loss: 6.4912 - mae: 1.6620
Epoch 38/100
303/303 [==============================] - 1s 3ms/sample - loss: 6.5521 - mae: 1.6329
Epoch 39/100
303/303 [=

In [7]:
# 結果を確認
all_scores

[2.4856298, 2.6586313, 2.6390946, 2.4644632]

In [8]:
# 結果の平均を確認
np.mean(all_scores)

2.5619547

In [ ]:
# フォールドごとに検証ログを保存
# k分割交差検証
k = 4
num_val_samples = len(train_data) // k
num_epochs = 500
all_mae_histories = []

for i in range(k):
    print('processing fold #', i)
    
    # 検証データの準備: フォールドiのデータ
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    # 訓練データの準備: 残りのフォールドのデータ
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
    # Kerasモデルを構築(コンパイル済み)
    model = build_model()
    
    # モデルをサイレントモード（verbose=0）で適合
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, 
                        batch_size=1)
    # モデルを検証データで評価
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

processing fold # 0
Train on 303 samples, validate on 101 samples
Epoch 1/500
303/303 [==============================] - 2s 8ms/sample - loss: 232.8007 - mae: 11.5369 - val_loss: 38.3344 - val_mae: 4.2100
Epoch 2/500
303/303 [==============================] - 1s 4ms/sample - loss: 28.7952 - mae: 3.6332 - val_loss: 24.0393 - val_mae: 3.3939
Epoch 3/500
303/303 [==============================] - 1s 3ms/sample - loss: 20.0257 - mae: 2.9894 - val_loss: 15.8310 - val_mae: 2.7313
Epoch 4/500
303/303 [==============================] - 1s 3ms/sample - loss: 17.4964 - mae: 2.8227 - val_loss: 14.7405 - val_mae: 2.7595
Epoch 5/500
303/303 [==============================] - 1s 3ms/sample - loss: 15.8187 - mae: 2.7015 - val_loss: 15.6653 - val_mae: 2.5838
Epoch 6/500
303/303 [==============================] - 1s 3ms/sample - loss: 14.6798 - mae: 2.6470 - val_loss: 14.0532 - val_mae: 2.4106
Epoch 7/500
303/303 [==============================] - 1s 3ms/sample - loss: 13.8481 - mae: 2.4987 - val_loss:

303/303 [==============================] - 1s 4ms/sample - loss: 6.1856 - mae: 1.6529 - val_loss: 8.8811 - val_mae: 2.0673
Epoch 61/500
303/303 [==============================] - 1s 4ms/sample - loss: 6.2608 - mae: 1.6352 - val_loss: 8.4573 - val_mae: 2.0103
Epoch 62/500
303/303 [==============================] - 1s 4ms/sample - loss: 6.2285 - mae: 1.5788 - val_loss: 8.7451 - val_mae: 2.0733
Epoch 63/500
303/303 [==============================] - 1s 4ms/sample - loss: 6.1845 - mae: 1.6338 - val_loss: 10.2034 - val_mae: 2.4935
Epoch 64/500
303/303 [==============================] - 1s 5ms/sample - loss: 6.3514 - mae: 1.5807 - val_loss: 8.8439 - val_mae: 2.1050
Epoch 65/500
303/303 [==============================] - 1s 5ms/sample - loss: 6.0298 - mae: 1.5894 - val_loss: 10.1461 - val_mae: 2.4584
Epoch 66/500
303/303 [==============================] - 1s 5ms/sample - loss: 5.5324 - mae: 1.5716 - val_loss: 8.6916 - val_mae: 2.0613
Epoch 67/500
303/303 [==============================] - 1s 

303/303 [==============================] - 1s 5ms/sample - loss: 4.1921 - mae: 1.3569 - val_loss: 8.6568 - val_mae: 1.9890
Epoch 121/500
303/303 [==============================] - 1s 5ms/sample - loss: 4.0309 - mae: 1.4108 - val_loss: 9.0137 - val_mae: 2.0732
Epoch 122/500
303/303 [==============================] - 1s 4ms/sample - loss: 3.7327 - mae: 1.3535 - val_loss: 9.7641 - val_mae: 2.2229
Epoch 123/500
303/303 [==============================] - 1s 4ms/sample - loss: 4.0762 - mae: 1.4096 - val_loss: 8.9629 - val_mae: 2.1686
Epoch 124/500
303/303 [==============================] - 1s 4ms/sample - loss: 3.9005 - mae: 1.3620 - val_loss: 10.2528 - val_mae: 2.3609
Epoch 125/500
303/303 [==============================] - 1s 4ms/sample - loss: 4.1760 - mae: 1.4243 - val_loss: 9.1610 - val_mae: 2.2296
Epoch 126/500
303/303 [==============================] - 1s 4ms/sample - loss: 3.6939 - mae: 1.3147 - val_loss: 10.8835 - val_mae: 2.2869
Epoch 127/500
303/303 [==============================

303/303 [==============================] - 1s 3ms/sample - loss: 2.8610 - mae: 1.1881 - val_loss: 8.9197 - val_mae: 2.0223
Epoch 180/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.6243 - mae: 1.1648 - val_loss: 9.2315 - val_mae: 2.2042
Epoch 181/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.9042 - mae: 1.1913 - val_loss: 10.0101 - val_mae: 2.2041
Epoch 182/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.6560 - mae: 1.1502 - val_loss: 10.3205 - val_mae: 2.2009
Epoch 183/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.7854 - mae: 1.2033 - val_loss: 9.5509 - val_mae: 2.1076
Epoch 184/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.6788 - mae: 1.1772 - val_loss: 8.9362 - val_mae: 2.1399
Epoch 185/500
303/303 [==============================] - 1s 3ms/sample - loss: 3.0483 - mae: 1.2028 - val_loss: 11.3648 - val_mae: 2.4186
Epoch 186/500
303/303 [=============================

303/303 [==============================] - 1s 3ms/sample - loss: 2.1937 - mae: 1.0420 - val_loss: 8.8823 - val_mae: 2.0765
Epoch 239/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.1725 - mae: 1.0148 - val_loss: 10.7181 - val_mae: 2.2335
Epoch 240/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.0753 - mae: 1.0678 - val_loss: 10.9175 - val_mae: 2.3681
Epoch 241/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.1705 - mae: 1.0737 - val_loss: 9.5014 - val_mae: 2.1907
Epoch 242/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.0510 - mae: 1.0623 - val_loss: 11.0058 - val_mae: 2.3027
Epoch 243/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.0136 - mae: 1.0629 - val_loss: 9.4675 - val_mae: 2.2071
Epoch 244/500
303/303 [==============================] - 1s 3ms/sample - loss: 1.8377 - mae: 1.0115 - val_loss: 10.9440 - val_mae: 2.3135
Epoch 245/500
303/303 [============================

...

303/303 [==============================] - 1s 3ms/sample - loss: 2.1316 - mae: 1.0550 - val_loss: 14.8225 - val_mae: 2.8676
Epoch 217/500
303/303 [==============================] - 1s 3ms/sample - loss: 1.9978 - mae: 0.9899 - val_loss: 15.0040 - val_mae: 2.7695
Epoch 218/500
303/303 [==============================] - 1s 3ms/sample - loss: 1.9784 - mae: 0.9853 - val_loss: 12.2879 - val_mae: 2.4182
Epoch 219/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.0900 - mae: 1.0261 - val_loss: 13.2649 - val_mae: 2.6147
Epoch 220/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.0566 - mae: 0.9983 - val_loss: 14.8109 - val_mae: 2.6807
Epoch 221/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.0338 - mae: 0.9752 - val_loss: 15.5193 - val_mae: 2.8287
Epoch 222/500
303/303 [==============================] - 1s 3ms/sample - loss: 2.2222 - mae: 1.0354 - val_loss: 13.4470 - val_mae: 2.5503
Epoch 223/500
303/303 [=========================

303/303 [==============================] - 1s 4ms/sample - loss: 1.5142 - mae: 0.8595 - val_loss: 14.8162 - val_mae: 2.6362
Epoch 276/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.5277 - mae: 0.8634 - val_loss: 14.6120 - val_mae: 2.6890
Epoch 277/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.4761 - mae: 0.8619 - val_loss: 14.5355 - val_mae: 2.7603
Epoch 278/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.3663 - mae: 0.8091 - val_loss: 14.5411 - val_mae: 2.7128
Epoch 279/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.5975 - mae: 0.9111 - val_loss: 13.7317 - val_mae: 2.6558
Epoch 280/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.3647 - mae: 0.8307 - val_loss: 15.3319 - val_mae: 2.9239
Epoch 281/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.3847 - mae: 0.8259 - val_loss: 14.0636 - val_mae: 2.6775
Epoch 282/500
303/303 [=========================

303/303 [==============================] - 1s 4ms/sample - loss: 1.2419 - mae: 0.8235 - val_loss: 13.9539 - val_mae: 2.7174
Epoch 335/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.1694 - mae: 0.7710 - val_loss: 13.9075 - val_mae: 2.5719
Epoch 336/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.2629 - mae: 0.8499 - val_loss: 13.5093 - val_mae: 2.5612
Epoch 337/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.3857 - mae: 0.8216 - val_loss: 13.6941 - val_mae: 2.5782
Epoch 338/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.3178 - mae: 0.8252 - val_loss: 12.8350 - val_mae: 2.5654
Epoch 339/500
303/303 [==============================] - 2s 5ms/sample - loss: 1.1629 - mae: 0.7903 - val_loss: 13.5030 - val_mae: 2.5804
Epoch 340/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.1584 - mae: 0.7743 - val_loss: 13.1969 - val_mae: 2.4947
Epoch 341/500
303/303 [=========================

303/303 [==============================] - 1s 3ms/sample - loss: 1.0355 - mae: 0.7528 - val_loss: 15.5226 - val_mae: 2.7444
Epoch 394/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.0159 - mae: 0.7182 - val_loss: 16.2944 - val_mae: 2.8606
Epoch 395/500
303/303 [==============================] - 1s 4ms/sample - loss: 0.9948 - mae: 0.7344 - val_loss: 14.7370 - val_mae: 2.6291
Epoch 396/500
303/303 [==============================] - 1s 4ms/sample - loss: 0.9762 - mae: 0.7268 - val_loss: 16.4517 - val_mae: 2.8352
Epoch 397/500
303/303 [==============================] - 1s 4ms/sample - loss: 0.9571 - mae: 0.7010 - val_loss: 14.9569 - val_mae: 2.7720
Epoch 398/500
303/303 [==============================] - 1s 5ms/sample - loss: 1.0885 - mae: 0.7707 - val_loss: 15.1837 - val_mae: 2.6731
Epoch 399/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.0146 - mae: 0.7326 - val_loss: 15.8560 - val_mae: 2.7746
Epoch 400/500
303/303 [=========================

303/303 [==============================] - 1s 3ms/sample - loss: 0.9162 - mae: 0.6847 - val_loss: 16.9827 - val_mae: 2.9438
Epoch 453/500
303/303 [==============================] - 1s 3ms/sample - loss: 0.8971 - mae: 0.6967 - val_loss: 16.1674 - val_mae: 2.7238
Epoch 454/500
303/303 [==============================] - 1s 4ms/sample - loss: 0.8847 - mae: 0.6922 - val_loss: 17.4869 - val_mae: 2.7640
Epoch 455/500
303/303 [==============================] - 1s 4ms/sample - loss: 0.9088 - mae: 0.7056 - val_loss: 16.5106 - val_mae: 2.7355
Epoch 456/500
303/303 [==============================] - 1s 4ms/sample - loss: 0.9096 - mae: 0.6755 - val_loss: 15.5635 - val_mae: 2.7369
Epoch 457/500
303/303 [==============================] - 1s 4ms/sample - loss: 1.0086 - mae: 0.7090 - val_loss: 16.6690 - val_mae: 2.8734
Epoch 458/500
303/303 [==============================] - 1s 4ms/sample - loss: 0.7542 - mae: 0.6393 - val_loss: 18.8550 - val_mae: 2.8747
Epoch 459/500
303/303 [=========================

In [ ]:
# k分割交差検証の平均スコアの履歴を構築
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
# 検証スコアのプロット
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
# 最初の１０個のデータ点を除外した検証スコアのプロット
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
# 最終的なモデルの訓練
model = build_model()

# 全データを使って訓練
model.fit(train_data, train_targets,
         epochs=80, batch_size=16)

# テストデータでの検証スコアを取得
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
# 最終的なスコアを確認
test_mae_score